# Check Execution Nodes

In [1]:
import json
import boto3
import requests
from requests.auth import HTTPBasicAuth

networks = {
    "goerli": {
        "requires_auth": True
    }
}

for network, requires_auth in networks.items():

    url = f"https://nodes.casimir.co/eth/{network}"

    headers = {'content-type': 'application/json'}
    payload = {
        "method": "eth_syncing",
        "params": [],
        "jsonrpc": "2.0",
        "id": 0,
    }

    auth = None
    if requires_auth:
        client = boto3.client('secretsmanager')
        response = client.get_secret_value(SecretId='casimir-rpc-credentials')
        secret = json.loads(response['SecretString'])
        auth = HTTPBasicAuth(secret['username'], secret['password'])

    response = requests.post(url, data=json.dumps(payload), headers=headers, auth=auth)

    if response.status_code != 200:
        print(f"Casimir {network} execution node is not responding")
    else: 
        data = response.json()
        if data['result'] == False:
            print(f"Casimir {network} execution node is fully synced")
        else:
            print(f"Casimir {network} execution node is syncing. Current block: {data['result']['currentBlock']}. Highest block: {data['result']['highestBlock']}")



Casimir goerli execution node is fully synced
